# Normalized Estimation for Homography Matrix

Homography matrixをnormalized estimationで求める手順のメモ

In [ ]:
%matplotlib inline
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt

def draw_matches(img1, img2, pts1, pts2, flag=None, mode="horizontal", thick=5):
    if flag is None:
        assert(pts1.shape[0] == pts2.shape[0] and pts1.shape[1] == pts2.shape[1] == 2)
    else:
        assert(pts1.shape[0] == pts2.shape[0] == len(flag) and pts1.shape[1] == pts2.shape[1] == 2)
    assert(mode == "horizontal" or mode == "vertical")
    
    num_pts = pts1.shape[0]
    rows1 = img1.shape[0]
    cols1 = img1.shape[1]
    rows2 = img2.shape[0]
    cols2 = img2.shape[1]

    if mode == "horizontal":
        out_img = np.zeros((max([rows1, rows2]), cols1 + cols2, 3), dtype="uint8")
        out_img[:rows1, :cols1, :] = cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
        out_img[:rows2, cols1:cols1 + cols2, :] = cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)
    elif mode == "vertical":
        out_img = np.zeros((rows1 + rows2, max([cols1, cols2]), 3), dtype="uint8")
        out_img[:rows1, :cols1, :] = cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
        out_img[rows1:rows1 + rows2, :cols1, :] = cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)

    for i in range(num_pts):
        if flag is not None and bool(flag[i]) == False:
            continue
        (x1, y1) = (pts1[i, 0], pts1[i, 1])
        (x2, y2) = (pts2[i, 0], pts2[i, 1])

        color = [random.randint(0, 255) for i in range(3)]

        if mode == "horizontal":
            cv2.circle(out_img, (int(x1), int(y1)), 4, color, 1)
            cv2.circle(out_img, (int(x2) + cols1, int(y2)), 4, color, 1)
            cv2.line(out_img, (int(x1), int(y1)), (int(x2) + cols1, int(y2)), color, thick)
        elif mode == "vertical":
            cv2.circle(out_img, (int(x1), int(y1)), 4, color, 1)
            cv2.circle(out_img, (int(x2), int(y2) + rows1), 4, color, 1)
            cv2.line(out_img, (int(x1), int(y1)), (int(x2), int(y2) + rows1), color, thick)

    return out_img

画像を読み込む

In [ ]:
img1 = cv2.imread("./images/perspective/1.JPG", cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread("./images/perspective/3.JPG", cv2.IMREAD_GRAYSCALE)
fig, ax = plt.subplots(1, 2, figsize=(17, 10))
ax[0].imshow(img1, cmap=plt.get_cmap("gray"))
ax[1].imshow(img2, cmap=plt.get_cmap("gray"))

SIFT/ORB/AKAZEで特徴点対応を推定

In [ ]:
feature_type = "ORB"
if feature_type == "SIFT":
    detector = cv2.xfeatures2d.SIFT_create()
    lowe_ratio = 0.6
    norm_type = cv2.NORM_L2
elif feature_type == "ORB":
    detector = cv2.ORB_create()
    detector.setMaxFeatures(4000)
    detector.setNLevels(8)
    detector.setScaleFactor(1.2)
    detector.setScoreType(cv2.ORB_FAST_SCORE)
    detector.setWTA_K(2)
    lowe_ratio = 0.8
    norm_type = cv2.NORM_HAMMING
elif feature_type == "AKAZE":
    detector = cv2.AKAZE_create()
    lowe_ratio = 0.8
    norm_type = cv2.NORM_HAMMING
else:
    raise ValueError("Undefined feature type: {}".format(feature_type))

# 特徴点と特徴量を抽出
kp1, desc1 = detector.detectAndCompute(img1, None)
kp2, desc2 = detector.detectAndCompute(img2, None)

# 第2近傍点まで探索する (k=2)
bf = cv2.BFMatcher(norm_type)
matches = bf.knnMatch(desc1, desc2, k=2)

# loweのratio testを行う
good_matches = []
for m, n in matches:
    if m.distance < lowe_ratio * n.distance:
        good_matches.append(m)

# good matchのみ取り出す
pts1 = [[kp1[m.queryIdx].pt[0], kp1[m.queryIdx].pt[1]] for m in good_matches]
pts1 = np.array(pts1)
pts2 = [[kp2[m.trainIdx].pt[0], kp2[m.trainIdx].pt[1]] for m in good_matches]
pts2 = np.array(pts2)

# 表示
plt.figure(figsize=(17, 10))
plt.imshow(draw_matches(img1, img2, pts1, pts2), cmap=plt.get_cmap("gray"))

## OpenCVでHomography matrixを推定

In [ ]:
H, inliers = cv2.findHomography(pts1, pts2, method=cv2.RANSAC, ransacReprojThreshold=(1.0 * np.sqrt(5.99)))
print(H)
print("inlier ratio: {} / {} = {}".format(np.count_nonzero(inliers), len(good_matches), np.count_nonzero(inliers) / len(good_matches)))

In [ ]:
plt.figure(figsize=(17, 10))
plt.imshow(draw_matches(img1, img2, pts1, pts2, inliers), cmap=plt.get_cmap("gray"))

## normalized estimationでHomography matrixを推定

参考: https://www.cs.ubc.ca/grads/resources/thesis/May09/Dubrofsky_Elan.pdf

原点を特徴点点群の重心に持っていき，重心から各対応点までの平均距離を$\sqrt{2}$にする．

In [ ]:
def normalize(pts):
    assert(pts.ndim == 2 and (pts.shape[1] == 2 or pts.shape[1] == 3))
    # 同次座標にしておく
    if pts.shape[1] == 2:
        pts = np.insert(pts, 2, 1, axis=1)
    # 返却値を用意
    normalized_pts = np.ones(shape=(pts.shape[0], 3))
    # 重心を計算
    centoroid = np.mean(pts, axis=0)
    # 重心を中心に持ってくる
    normalized_pts[:, 0] = pts[:, 0] - centoroid[0]
    normalized_pts[:, 1] = pts[:, 1] - centoroid[1]
    # 中心からの各点の距離を計算
    dists = np.sqrt(normalized_pts[:, 0] ** 2 + normalized_pts[:, 1] ** 2)
    # 平均をsart(2)にするscaleを計算
    scale = np.sqrt(2) / np.mean(dists)
    # transformationを計算
    transformation = np.array([[scale, 0, -scale * centoroid[0]],
                               [0, scale, -scale * centoroid[1]],
                               [0, 0, 1]])
    # 座標変換
    normalized_pts = transformation.dot(pts.T).T
    return normalized_pts, transformation

DLTでHomography matrixを求める．

In [ ]:
def find_homography_DLT(pts1, pts2):
    assert(pts1.shape == pts2.shape)
    num_pts = pts1.shape[0]
    assert(4 <= num_pts)
    # 行列Aを作成
    A = np.zeros(shape=(2 * num_pts, 9))
    ## [u, v, 1]
    A[:num_pts, 0:2] = pts1[:, 0:2]
    A[:num_pts, 2] = np.ones(shape=(num_pts))
    ## [-u, -v, -1]
    A[-num_pts:, 3:5] = -pts1[:, 0:2]
    A[-num_pts:, 5] = -np.ones(shape=(num_pts))
    ## [-u * u', -v * u', -u']
    A[:num_pts, 6] = -pts1[:, 0] * pts2[:, 0]
    A[:num_pts, 7] = -pts1[:, 1] * pts2[:, 0]
    A[:num_pts, 8] = -pts2[:, 0]
    ## [u * v', v * v', v']
    A[-num_pts:, 6] = pts1[:, 0] * pts2[:, 1]
    A[-num_pts:, 7] = pts1[:, 1] * pts2[:, 1]
    A[-num_pts:, 8] = pts2[:, 1]
    # SVD
    ## Sは固有値が降順に格納されている
    ## Vは転置されているので，Vの各行が固有ベクトル
    U, s, V = np.linalg.svd(A, full_matrices=True)
    # 最小固有値の固有ベクトルを並び替えたものがHomography matrixになる
    H = V[8, :].reshape((3, 3))
    # H[2, 2]が1になるように正規化
    H /= H[2, 2]
    return H

def find_homography_normalized_DLT(pts1, pts2):
    # 座標を正規化
    normalized_pts1, T1 = normalize(pts1)
    normalized_pts2, T2 = normalize(pts2)
    # DLTでHomography matrixを求める
    Hi = find_homography_DLT(normalized_pts1, normalized_pts2)
    # 座標変換
    H = np.linalg.inv(T2).dot(Hi).dot(T1)
    return H / H[2, 2]

Homography Matrixの評価にはsymmetric transfer errorを用いる．

In [ ]:
def evaluate_homography(H, pts1, pts2, sigma=1.0):
    assert(pts1.shape == pts2.shape)
    num_pts = pts1.shape[0]
    # 自由度2のカイ2乗値がinlierの閾値になる
    chi2_threshold = 5.991
    # 同次座標に変換
    if pts1.shape[1] == 2:
        pts1 = np.insert(pts1, 2, 1, axis=1)
    if pts2.shape[1] == 2:
        pts2 = np.insert(pts2, 2, 1, axis=1)
    # 画像1の特徴点を画像2上に写像 (同次座標に戻しておく)
    pts1_in_img2 = H.dot(pts1.T).T
    pts1_in_img2[:, 0] /= pts1_in_img2[:, 2]
    pts1_in_img2[:, 1] /= pts1_in_img2[:, 2]
    pts1_in_img2[:, 2] /= pts1_in_img2[:, 2]
    # 画像2の特徴点を画像1上に写像 (同次座標に戻しておく)
    pts2_in_img1 = np.linalg.inv(H).dot(pts2.T).T
    pts2_in_img1[:, 0] /= pts2_in_img1[:, 2]
    pts2_in_img1[:, 1] /= pts2_in_img1[:, 2]
    pts2_in_img1[:, 2] /= pts2_in_img1[:, 2]
    # 写像された点と本来の点との正規化L2ノルムを計算する
    dists2_in_img2 = np.linalg.norm((pts2[:, 0:2] - pts1_in_img2[:, 0:2]) / sigma, axis=1) ** 2
    dists2_in_img1 = np.linalg.norm((pts1[:, 0:2] - pts2_in_img1[:, 0:2]) / sigma, axis=1) ** 2
    # inlierを探す
    ## カイ2乗値以下であればであればinlierとする
    inliers_in_img2 = dists2_in_img2 <= chi2_threshold
    inliers_in_img1 = dists2_in_img1 <= chi2_threshold
    inliers = inliers_in_img2 & inliers_in_img1
    # inlierのみでscoreを計算する
    ## 正規化L2ノルムとカイ2乗値との差がスコアになる (高いほど良い)
    score = 0
    score += np.sum(chi2_threshold - dists2_in_img2[inliers_in_img2])
    score += np.sum(chi2_threshold - dists2_in_img1[inliers_in_img1])
    return score, inliers

RANSACでHomography matrixを求める．

In [ ]:
def find_homography_RANSAC(pts1, pts2, sigma=1.0, num_iterations=50):
    assert(pts1.shape == pts2.shape)
    num_pts = pts1.shape[0]
    # RANSACを回す
    best_score = 0
    best_inliers = np.full(num_pts, False)
    for _ in range(num_iterations):
        # ランダム抽出
        sample_indices = np.random.choice(num_pts, min(4, num_pts), False)
        sample_pts1 = pts1[sample_indices, :]
        sample_pts2 = pts2[sample_indices, :]
        # サンプル点でHomography matrixを求める
        H = find_homography_normalized_DLT(sample_pts1, sample_pts2)
        # scoreを計算
        score, inliers = evaluate_homography(H, pts1, pts2, sigma)
        # best modelを更新
        ## 条件A: np.sum(best_inliers) < np.sum(inliers)
        ## 条件B: best_score < score
        if np.sum(best_inliers) < np.sum(inliers):
            best_score = score
            best_inliers = inliers
    # inlierのみでHを推定
    H = find_homography_normalized_DLT(pts1[best_inliers], pts2[best_inliers])
    return H, best_inliers

OpenCVの関数と自作の関数で比較する．

In [ ]:
H, inliers = cv2.findHomography(pts1, pts2, method=cv2.RANSAC, ransacReprojThreshold=(1.0 * np.sqrt(5.991)), maxIters=200)
score, inliers = evaluate_homography(H, pts1, pts2, sigma=1.0)
print("OpenCV")
print(H)
print("score: {}, inlier ratio: {} / {} = {}".format(score, np.sum(inliers), len(good_matches), np.sum(inliers) / len(good_matches)))
print("")
plt.figure(figsize=(17, 10))
plt.imshow(draw_matches(img1, img2, pts1, pts2, inliers), cmap=plt.get_cmap("gray"))

H, inliers = find_homography_RANSAC(pts1, pts2, sigma=1.0, num_iterations=200)
score, inliers = evaluate_homography(H, pts1, pts2, sigma=1.0)
print("mine")
print(H)
print("score: {}, inlier ratio: {} / {} = {}".format(score, np.sum(inliers), len(good_matches), np.sum(inliers) / len(good_matches)))
print("")
plt.figure(figsize=(17, 10))
plt.imshow(draw_matches(img1, img2, pts1, pts2, inliers), cmap=plt.get_cmap("gray"))